In [1]:

from copy import deepcopy

import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import time

In [2]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *

In [3]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def get_params_amount(model):
    amount = 0
    for _, layer in model.named_children():
        if isinstance(layer, ExpandingLinear):
            for linear in layer.embed_linears:
                amount += linear.weight_values.shape[0]
            amount += layer.weight_values.shape[0]
        elif isinstance(layer, nn.Linear):
            amount += linear.in_features * linear.out_features
    return amount

In [5]:
def get_zero_params_amount(model, eps=1e-8):
    amount = 0
    for _, layer in model.named_children():
        if isinstance(layer, ExpandingLinear):
            for linear in layer.embed_linears:
                amount += linear.weight_values[linear.weight_values.abs() < eps].shape[0]
            amount += layer.weight_values[layer.weight_values.abs() < eps].shape[0]
        elif isinstance(layer, nn.Linear):
            amount += linear.weight[linear.weight.abs() < eps].numel()
    return amount

In [6]:
def get_to_replace_params_amount(ef, model, layer_names, mask, choose_threshold):
    chosen_edges = 0
    for layer_name in layer_names:
        chosen_edges += len(ef.choose_edges_threshold(model, layer_name, choose_threshold, mask)[0])
    return chosen_edges

In [7]:
def train_sparse_recursive(model, train_loader, val_loader, hyperparams):
    optimizer = optim.Adam(model.parameters(), lr=hyperparams['lr'])
    criterion = nn.CrossEntropyLoss()
    ef = EdgeFinder(hyperparams['metric'], val_loader, device, aggregation_mode='mean')

    replace_epoch = [0]
    val_losses = []
    len_choose = get_model_last_layer(model).count_replaces[0]
    for epoch in range(hyperparams['num_epochs']):
        t0 = time.time()
        model.train()
        train_loss = 0
        for i, (inputs, targets) in enumerate(tqdm(train_loader)):
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            train_loss += loss.item()
        
        # if len(replace_epoch) > 1:
        #     for g in optimizer.param_groups:  
                # g['lr'] *= 0.9
        
        train_loss /= len(train_loader)
        train_time = time.time() - t0

        model.eval()
        val_loss = 0
        all_targets = []
        all_preds = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_targets.extend(targets.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())

        val_loss /= len(val_loader)
        val_accuracy = accuracy_score(all_targets, all_preds)

        print(f"Epoch {epoch + 1}/{hyperparams['num_epochs']}, Train Loss: {train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        val_losses.append(val_loss)
        if len(val_losses) > hyperparams['window_size'] and epoch - replace_epoch[-1] > 8:
            recent_changes = [abs(val_losses[i] - val_losses[i - 1]) for i in range(-hyperparams['window_size'], 0)]
            avg_change = sum(recent_changes) / hyperparams['window_size']
            if avg_change < hyperparams['threshold']:
                # layer = model.fc0
                # mask = torch.ones_like(layer.weight_values, dtype=bool)
                # len_choose = edge_replacement_func_new_layer(layer, mask, optimizer, val_loader, metric, 0.3, 'mean')

                layer = model.fc0
                mask = torch.ones_like(layer.weight_values, dtype=bool)
                len_choose = edge_replacement_func_new_layer(model, 'fc0', mask, optimizer, hyperparams['choose_threshold'], ef)

                wandb.log({'len_choose': len_choose})
                replace_epoch += [epoch]

                # if len(replace_epoch) == 2:
                #     for g in optimizer.param_groups:
                #         g['lr'] *= 100


        params_amount = get_params_amount(model)
        # zero_params_amount = get_zero_params_amount(model)
        layer = model.fc0
        mask = torch.ones_like(layer.weight_values, dtype=bool)
        replace_params = get_to_replace_params_amount(ef, model, ['fc0'], mask, hyperparams['choose_threshold'])
        wandb.log({'val loss': val_loss, 'val accuracy': val_accuracy,
                   'train loss': train_loss, 'params amount': params_amount,
                   'params to replace amount': replace_params, 'train time': train_time,
                   'params ratio': (params_amount - replace_params) / params_amount,
                   'lr': optimizer.param_groups[0]['lr'], 'acc amount': val_accuracy / params_amount})

def edge_replacement_func_new_layer(model, layer_name, mask, optim, choose_threshold, ef):
    layer = model.__getattr__(layer_name)
    # ef = EdgeFinder(metric, val_loader, device, aggregation_mode)
    chosen_edges = ef.choose_edges_threshold(model, layer_name, choose_threshold, mask)
    print("Chosen edges:", chosen_edges, len(chosen_edges[0]))
    layer.replace_many(*chosen_edges)

    if len(chosen_edges[0]) > 0:
        optim.add_param_group({'params': layer.embed_linears[-1].weight_values})
        optim.add_param_group({'params': layer.weight_values})
    print(len(chosen_edges[0]))
    return len(chosen_edges[0])

In [8]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28, hidden_size=16):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, 10)
        # self.fc1 = nn.Linear(hidden_size, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc0(x)
        return x

In [9]:
# Dataset and Dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Load dataset and split into train/validation sets
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [10]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0])

In [11]:
hyperparams = {
    "num_epochs": 64,
    "metric": MagnitudeL2Metric(nn.CrossEntropyLoss()), 
    "aggregation_mode": "mean",  
    "choose_threshold": 0.5,  
    "threshold": 0.05,
    "window_size": 5,  
    "lr": 1e-4, 
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

'num_epochs: 64, metric: MagnitudeL2Metric, aggregation_mode: mean, choose_threshold: 0.5, threshold: 0.05, window_size: 5, lr: 0.0001'

In [12]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vanyamironov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [13]:
run = wandb.init(
    project="self-expanding-nets",
    name=f"trash",
)

In [14]:
train_sparse_recursive(sparse_model, train_loader, val_loader, hyperparams)

100%|██████████| 750/750 [00:03<00:00, 236.67it/s]


Epoch 1/64, Train Loss: 1.4805, Val Loss: 0.9977, Val Accuracy: 0.8192


100%|██████████| 750/750 [00:02<00:00, 303.16it/s]


Epoch 2/64, Train Loss: 0.8088, Val Loss: 0.6843, Val Accuracy: 0.8528


100%|██████████| 750/750 [00:02<00:00, 288.28it/s]


...

Epoch 62/64, Train Loss: 0.1533, Val Loss: 0.1826, Val Accuracy: 0.9468


100%|██████████| 750/750 [00:04<00:00, 163.52it/s]


Epoch 63/64, Train Loss: 0.1516, Val Loss: 0.1811, Val Accuracy: 0.9477


100%|██████████| 750/750 [00:04<00:00, 163.65it/s]


Epoch 64/64, Train Loss: 0.1504, Val Loss: 0.1812, Val Accuracy: 0.9473
Chosen edges: tensor([[  1,   3,   3,   3,   4,   6,   6,   7,   9,   9,   9,   9,   9,   0,
           0,   4,   4,   4,   2,   2,   3,   3,   3,   4,   7,   7,   7,   9,
           5,   5,   2,   2,   6,   7,   8,   8,   9,   2,   7,   7,   8,   8,
           8,   9,   9,   9,   4,   5,   5,   6,   6,   3,   3,   8,   9,   9],
        [304, 193, 304, 585, 737, 710, 719, 122,  70,  95, 101, 104, 164, 784,
         785, 828, 830, 896, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956,
         957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970,
         971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984]]) 56
56


- прунинг по метрике на следующей эпохе после реплейса
